## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges</span>
<sub>*And setting constants

In [ ]:
import re
import sys
import json
import warnings
import requests
import cloudscraper
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

# from fp.fp                            import FreeProxy
from itertools                          import count
from bs4                                import BeautifulSoup
from selenium                           import webdriver
from google.cloud                       import bigquery
from urllib.request                     import urlopen
from selenium.webdriver.common.by       import By
from scrapy.utils.log                   import configure_logging
from selenium.webdriver.common.keys     import Keys
from IPython.display                    import set_matplotlib_formats
from selenium.webdriver.chrome.service  import Service
from selenium.webdriver.chrome.options  import Options
from selenium.webdriver.support.ui      import WebDriverWait
from webdriver_manager.chrome           import ChromeDriverManager
from selenium.webdriver.support         import expected_conditions as EC

In [ ]:
%matplotlib inline
set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

sys.path.insert(1, '../my_encrypter')
import encrypt

encrypt.decrypt_json_file('../credentials.json')

MAX_PAGES :int  = 15
COLORS    :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS  :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                'uint16', 'uint32', 'uint64']

In [ ]:
data_jobs_titles = ['Data entry', 'Data engineer',
                    'Data scientist', 'Data analyst',
                    'ML developer']

# indeed_countries = ['de', 'uk', 'www']# USACOLORS    :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS  :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                   'uint16', 'uint32', 'uint64'] don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']

## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [4]:
def scrape_page_fast(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay= 10,browser= {
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False}) 
    
    content = scraper.get(url).text 
    return BeautifulSoup(content)



def scrape_page(url: str, retrieve_new_url= None, user_agent= None) -> BeautifulSoup:
    
    # chrome_options = Options()
    # chrome_options.add_argument('--headless')  # Optional: Run Chrome in headless mode
    # chrome_options.add_argument(f"--user-agent={ua.random}")

    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


    html = BeautifulSoup(driver.page_source)

    if retrieve_new_url:
        new_url = driver.current_url

        driver.quit()
        return html, new_url
        
    driver.quit()
    return html

In [5]:
# Comparing the two scrapers speed.
%timeit scrape_page_fast('https://linkedin.com')
%timeit scrape_page('https://linkedin.com')

# We can find that there's about 10X speed for the `scrape_page_fast` 🤯.

1.55 s ± 356 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
13 s ± 2.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# TODO

# def indeed_scraper(country   : str,
#                    job_title : str
#                    page      : int) -> BeautifulSoup:

#     ua           : str = UserAgent.random
#     modefid_page : int = (int(page) - 1) * 10
#     headers      : dict = {
#         'User-Agent': ua,
#         'Accept-Encoding': '*',
#         'Connection': 'keep-alive'}
        
#     url          : str = f'https://{country}.indeed.com/jobs?q={job_title}&start={modefid_page}'

#     response = requests.get(url, headers= headers)
#     soup = BeautifulSoup(response.content,'html.parser')

#     return  soup

In [7]:
def linkedin_scraper(country:   str, # This scraper scrapes all pages at once
                     job_title: str,
                     page:      int) -> BeautifulSoup:
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
        'Accept-Encoding': '*',
        'Connection': 'keep-alive'}

    page: int = (page - 1) * 25
    url: str = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={job_title}&' + \
    f'location={country}&geoId=90000084&trk=public_jobs_j%20obs-search-bar_search-submit&position=1&pageNum=0&start={page}'

    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.content,'html.parser')

    return  soup

## <center><strong>Collecting<span style= 'color: #4895e0'> jobs </span> data

**What we will do in this section:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [8]:
def convert_to_integer(number_string):

    cleaned_string = ''.join(filter(str.isdigit, number_string))
    cleaned_string = cleaned_string.rstrip('+')
    integer_value = int(cleaned_string)

    return integer_value

In [9]:
stacked_df: dict = {'soup':        [],
                    'job_title':   [],
                    'country':     [],
                    'platform':    [],
                    'total_jobs':  []}

print('Starting LinkedIn scraper.')

for country in linkedin_countries:

    print(f'Loading the {country} data.')
    for job_title in data_jobs_titles:
        
        full_soup   : str = ''
        total_jobs  : list = convert_to_integer(scrape_page(f'https://www.linkedin.com/jobs/search?keywords={job_title}&location={country}&' + 
                                                            f'\geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0').find(
                                                                   'span', 'results-context-header__job-count').text)

        for i in count(0):
            page = i + 1
            page_soup: BeautifulSoup = linkedin_scraper(country= country,
                                                        job_title= job_title,
                                                        page= i + page)
            full_soup += str(page_soup)
            full_soup += ' <br> '

            if (page_soup.find_all('li') is None) or (page == MAX_PAGES):
                break

        stacked_df['platform']      .append('LinkedIn')
        stacked_df['total_jobs']    .append(total_jobs)
        stacked_df['soup']          .append(full_soup)
        stacked_df['job_title']     .append(job_title)
        stacked_df['country']       .append(country)

        print(f'Finished loading {job_title} jobs from the {country}.')
    print('\n')

Starting LinkedIn scraper.
Loading the European Union data.
Finished loading Data entry jobs from the European Union.
Finished loading Data engineer jobs from the European Union.
Finished loading Data scientist jobs from the European Union.
Finished loading Data analyst jobs from the European Union.
Finished loading ML developer jobs from the European Union.


Loading the United States data.
Finished loading Data entry jobs from the United States.
Finished loading Data engineer jobs from the United States.
Finished loading Data scientist jobs from the United States.
Finished loading Data analyst jobs from the United States.
Finished loading ML developer jobs from the United States.




In [10]:
# Indeed scraping TODO
# for country in indeed_countries:
#     for job_title in data_jobs_titles:
#         stacked_pages_soup: str = ''

#         for i in count(0):

#             try:
#                 page = i + 1
#                 soup: BeautifulSoup = indeed_scraper(country= country,
#                                                      job_title= job_title,
#                                                      page= page)

#                 stacked_pages_soup += '\n<br> ' + str(soup)
#                 if page > max_page:
#                     print('Finished loading current country or job title.\n')
#                     break

#             except KeyError as e:
#                 print('Finished loading current country or job title.\n')
#                 break

#         stacked_df['platform'].append('Indeed')
#         stacked_df['soup'].append(stacked_pages_soup)
#         stacked_df['job_title'].append(job_title)
#         stacked_df['country'].append(country)

#### **Collecting jobs & companies basic data**

In [11]:
stacked_df = pd.DataFrame(stacked_df)

stacked_df['jobs_locations']  = None
stacked_df['listing_dates']   = None
stacked_df['jobs_titles']     = None
stacked_df['companies_names'] = None

stacked_df['soups']           = None
stacked_df['jobs_links']      = None


# Collecting linkedIn jobs links
for i, soup in enumerate(stacked_df[stacked_df['platform'] == 'LinkedIn']['soup']):

    soup = BeautifulSoup(soup)
    jobs = soup.find_all('div', 
           'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')

    linkedin_jobs_locations       = []
    linkedin_jobs_listing_dates   = []
    linkedin_jobs_titles          = []
    linkedin_jobs_companies_names = []
    linkedin_jobs_links           = []
    linkedin_companies_links      = []
    
    for job in jobs:

        try: linkedin_jobs_listing_dates               .append(job.find('time', 'job-search-card__listdate')['datetime'])
        except TypeError:  linkedin_jobs_listing_dates .append(np.nan)
        
        linkedin_jobs_locations            .append(job.find('span', 'job-search-card__location').text.strip())
        linkedin_jobs_titles               .append(job.find('h3', 'base-search-card__title').text.strip())
        linkedin_jobs_companies_names      .append(job.find('a', 'hidden-nested-link').text.strip())
        linkedin_companies_links           .append(job.find('a', 'hidden-nested-link')['href'])
        linkedin_jobs_links                .append(job.find
                                                   ('a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')['href'])

    # I will use sets and tuples instead of lists because they are faster.
    stacked_df['jobs_locations']    [i] = tuple(linkedin_jobs_locations)
    stacked_df['listing_dates']     [i] = tuple(linkedin_jobs_listing_dates)
    stacked_df['jobs_titles']       [i] = tuple(linkedin_jobs_titles)
    stacked_df['companies_names']   [i] = tuple(linkedin_jobs_companies_names)

    stacked_df['jobs_links']        [i] = tuple(linkedin_jobs_links)
    stacked_df['soups']             [i] = tuple(linkedin_companies_links)

In [12]:
# Collecting indeed jobs links TOFIX
# for i, soup in enumerate(stacked_df[stacked_df['platform'] == 'Indeed']['soup']):
    
#     indeed_jobs_links: list = []
#     indeed_companies_links: list = []

#     for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
#         job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
#         indeed_jobs_links.append(job_link)

#     for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
#         company_link = BeautifulSoup(scrape_page(job_link)).find('a', 'css-775knl emf9s7v0')
#         indeed_companies_links.append(company_link)
#         print(company_link)
        
#     stacked_df['total_jobs'][i] = len(indeed_jobs_links)
#     stacked_df['jobs_links'][i] = indeed_jobs_links
#     stacked_df['soups'][i] = indeed_companies_links

#### **Loading more jobs data**

In [14]:
# Collecting more LinkedIn data

# We will collect the: 
#    describtion, credential category, location type

stacked_df['describtion']         = None
stacked_df['reqierd_credential']  = None
stacked_df['location_type']       = None
stacked_df['employment_type']     = None
stacked_df['industry']            = None

for i, jobs in enumerate(stacked_df['jobs_links']): # TOREMOVE remove the loop limit

    describtions:            list = []
    reqierd_credentials:     list = []
    location_types:          list = []
    employment_types:        list = []
    industries:              list = []
    
    for job in jobs:

        try:
            job_data:  dict = json.loads(scrape_page_fast(job).find(
                'script', attrs= {'type': 'application/ld+json'}).text)
        except:
            job_data = np.nan

        # Describtion
        try:      describtions         .append(job_data['description'].strip())
        except:   describtions         .append(np.nan)

        # Reqierd credential
        try:      reqierd_credentials  .append(job_data['educationRequirements']['credentialCategory'])
        except:   reqierd_credentials  .append(np.nan)

        # Employment type
        try:      employment_types     .append(job_data['employmentType'].strip())
        except:   employment_types     .append(np.nan)

        # Location type
        try:      location_types       .append(job_data['jobLocationType'])
        except:   location_types       .append(np.nan)

        # Industry
        try:      industries           .append(job_data['industry'].strip())
        except:   industries           .append(np.nan)

    # Now we are going to convert list into tuples.
    stacked_df['describtion']          [i] = tuple(describtions)
    stacked_df['reqierd_credential']   [i] = tuple(reqierd_credentials)
    stacked_df['location_type']        [i] = tuple(location_types)
    stacked_df['employment_type']      [i] = tuple(employment_types)
    stacked_df['industry']             [i] = tuple(industries)

In [16]:
stacked_df.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,soups,jobs_links,describtion,reqierd_credential,location_type,employment_type,industry
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,European Union,LinkedIn,264000,"(San Jose, CA, Belmont, CA, San Jose, CA, Stoc...","(nan, 2023-07-14, 2023-07-22, 2023-07-20, 2023...",(Data Entry Operator Junior / Part Time (Remot...,"(Seekmate, Get.It Recruit - Administrative, Cu...",(https://www.linkedin.com/company/seekmate?trk...,(https://www.linkedin.com/jobs/view/data-entry...,(This is a remote position.&lt;p&gt;&lt;br&gt;...,"(high school, high school, high school, high s...","(TELECOMMUTE, TELECOMMUTE, TELECOMMUTE, TELECO...","(PART_TIME, FULL_TIME, FULL_TIME, FULL_TIME, F...","(Computer Networking Products, Human Resources..."
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,European Union,LinkedIn,39000,"(San Francisco, CA, San Francisco, CA, San Fra...","(2023-04-07, 2023-04-28, 2023-03-29, 2023-05-2...","(Data Engineer, Data Engineer, Data Engineer, ...","(Patreon, SEPHORA, Laguna Games, Applied Intui...",(https://www.linkedin.com/company/patreon?trk=...,(https://www.linkedin.com/jobs/view/data-engin...,(Patreon is the best place for creators to bui...,"(bachelor degree, bachelor degree, bachelor de...","(nan, nan, nan, nan, TELECOMMUTE, nan, TELECOM...","(FULL_TIME, FULL_TIME, FULL_TIME, FULL_TIME, F...","(Technology, Information and Internet, Persona..."
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,European Union,LinkedIn,142000,"(San Francisco, CA, Palo Alto, CA, Emeryville,...","(2023-06-16, 2023-07-10, 2023-05-01, 2023-07-0...","(Data Scientist, Data Scientist, Machine Learn...","(Experfy, Glean, Liminal, Qualia, Tavus, Softw...",(https://www.linkedin.com/company/experfy?trk=...,(https://www.linkedin.com/jobs/view/data-scien...,(The successful candidate will have a strong b...,"(bachelor degree, bachelor degree, bachelor de...","(nan, nan, nan, nan, TELECOMMUTE, nan, TELECOM...","(FULL_TIME, FULL_TIME, FULL_TIME, FULL_TIME, F...","(Technology, Information and Internet, Technol..."
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,European Union,LinkedIn,156000,"(San Francisco, CA, Cupertino, CA, Cupertino, ...","(2023-02-20, 2023-05-11, 2023-05-08, 2023-01-2...","(Data Analyst, Data Analyst III, Data Analyst ...","(Digital Janet, WinMax, WinMax, ExaTech Inc, F...",(https://www.linkedin.com/company/digitaljanet...,(https://www.linkedin.com/jobs/view/data-analy...,(&lt;strong&gt;Title: Data Analyst&lt;br&gt;&l...,"(bachelor degree, bachelor degree, bachelor de...","(nan, nan, nan, nan, nan, nan, nan, TELECOMMUT...","(FULL_TIME, CONTRACTOR, CONTRACTOR, CONTRACTOR...","(Staffing and Recruiting, Staffing and Recruit..."
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,European Union,LinkedIn,989,"(San Francisco, CA, Mountain View, CA, San Fra...","(2023-06-23, 2023-07-12, 2023-06-05, 2023-07-2...","(Software Engineer - AI/ML, Machine Learning S...","(Descript, Enterprise Minds, Inc, Sieve, Auror...",(https://www.linkedin.com/company/descript?trk...,(https://www.linkedin.com/jobs/view/software-e...,(Our vision is to build the next generation pl...,"(bachelor degree, nan, bachelor degree, bachel...","(TELECOMMUTE, nan, nan, nan, nan, nan, nan, na...","(FULL_TIME, nan, FULL_TIME, FULL_TIME, CONTRAC...","(Technology, Information and Internet, nan, So..."


## <center><strong><span style= 'color: #517afc'>Saving</span> the data

In [18]:
linkedin_dfs: list = [] # We will group all rows as dfs to stack them later.

for _, row in stacked_df.iterrows():
    del _

    df = pd.DataFrame({
        'jobs_locations'      : row['jobs_locations'],
        'listing_dates'       : row['listing_dates'],
        'jobs_titles'         : row['jobs_titles'],
        'companies_names'     : row['companies_names'],
        'jobs_links'          : row['jobs_links'],
        'describtion'         : row['describtion'],
        'location_type'       : row['location_type'],
        'employment_type'     : row['employment_type'],
        'industry'            : row['industry'],
        'reqierd_credential'  : row['reqierd_credential']})

    df['country']     = row['country']
    df['job_title']   = row['job_title']
    df['total_jobs']  = row['total_jobs'] # This column indicates total jobs grouped by country & job title.

    linkedin_dfs.append(df)

linckedin_df = pd.concat(linkedin_dfs, ignore_index= True, sort= False)
linckedin_df.to_parquet(r'data/linckedin_jobs.parquet')
linckedin_df.to_csv(r'data/linckedin_jobs.csv')

encrypt.encrypt_json_file('../credentials.json')